# User-based Collaborative Filtering

Import Packages

In [1]:
import math
import numpy as np
from numpy import linalg as LA
from scipy.sparse import coo_matrix
import pandas as pd
from IPython.display import HTML, display

## Read Data: movies and ratings

Read Movies and Define `displayMovies`

In [2]:
movies = pd.read_csv('movies_w_imgurl.csv')

In [3]:
def displayMovies(movieIds, ratings=[]):
    i = 0
    html = ""
    for movieId in movieIds:
        mov = movies[movies['movieId'] == movieId].iloc[0]        
        html += "<div style='display:inline-block;min-width:150px;max-width:150px;vertical-align: top;'>"
        html += "<img src='%s' width='120'><br/>" % mov.imgurl
        if i < len(ratings):
            html += "<span>%.4f</span><br/>" % ratings[i]
        html += "%s<br/>" % mov.title
        if mov.genres != '':
            ul = "<ul>"
            for genre in mov.genres.split('|'):
                ul += "<li>%s</li>" % genre
            ul += "</ul>"
            html += "%s<br/>" % ul
        html += "</div>"
        i += 1
    display(HTML(html))

Read Rating Data

In [4]:
ratings = pd.read_csv('ratings-9_1.csv')
train = ratings[ratings['type'] == 'train'][['userId', 'movieId', 'rating']]
test = ratings[ratings['type'] == 'test'][['userId', 'movieId', 'rating']]

In [5]:
def displayLikedUserMovies(userId, topK):
    display(HTML("<h3>%s</h3><hr>" % userId))
    topKRatings = train[train['userId'] == userId].sort_values(by='rating', ascending=False).head(topK)
    displayMovies(topKRatings.movieId.values, topKRatings.rating.values)

## Convert Ratings to User-Item Sparse Matrix

### Create Index to Id Maps

In [6]:
movieIdToIndex = {}
indexToMovieId = {}
colIdx = 0
for movieId in movies.movieId:
    movieIdToIndex[movieId] = colIdx
    indexToMovieId[colIdx] = movieId
    colIdx += 1

In [7]:
userIdToIndex = {}
indexToUserId = {}
rowIdx = 0
for userId in ratings.userId.unique():
    userIdToIndex[userId] = rowIdx
    indexToUserId[rowIdx] = userId
    rowIdx += 1

### Creat User-Item Sparse Matrix

In [8]:
rows = []
cols = []
vals = []
for row in ratings.itertuples():
    rows.append(userIdToIndex[row.userId])
    cols.append(movieIdToIndex[row.movieId])
    vals.append(row.rating)
coomat = coo_matrix((vals, (rows, cols)), shape=(rowIdx, colIdx))

## Compute User-User Similarities

Compute $l_2$-norm

In [9]:
norms = LA.norm(coomat.toarray(), ord=2, axis=1)

Normalize Row Vectors

In [10]:
np.seterr(divide='ignore', invalid='ignore')
normmat = np.divide(coomat.transpose().toarray(), norms).T

In [11]:
np.nan_to_num(normmat, 0.0)

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.17996851,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.11684125,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

Compute Similarities ( = inner product)

In [12]:
sims = pd.DataFrame(data=np.matmul(normmat, normmat.T), index=ratings.userId.unique(), columns=ratings.userId.unique())

## Similarity Example

In [13]:
topK = 6
userId = 33
simUsers = sims.loc[33, :].sort_values(ascending=False).head(11).tail(5)

In [14]:
displayLikedUserMovies(userId, topK)
for index, simUser in simUsers.iteritems():
    displayLikedUserMovies(index, topK)

## User Rating Prediction

In [15]:
userId = 33

### Predict Ratings

In [16]:
ratingDF = pd.DataFrame(data=coomat.toarray(), index=ratings.userId.unique(), columns=movies.movieId.values)
binDF = ratingDF.applymap(lambda x: math.ceil(x/10))

In [17]:
userAvgRatings = pd.DataFrame(data=ratingDF.sum(axis=1).divide(binDF.sum(axis=1)), columns=['avg'])

In [18]:
simUsers = sims.loc[userId, :]
simUsers[userId] = 0

In [27]:
simRatingSums = (ratingDF - binDF.T.multiply(userAvgRatings['avg']).T).T.multiply(simUsers).T).sum(axis=0)
simSums = binDF.T.multiply(simUsers).T.sum(axis=0)
recItemRatings = userAvgRatings.loc[userId].avg + pd.Series(data=simRatingSums.divide(simSums), name='prediction')
recItemRatings.fillna(0, inplace=True)

SyntaxError: invalid syntax (<ipython-input-27-c979074fb476>, line 1)

In [26]:
binDF.T.multiply(userAvgRatings['avg']).T

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.486842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.348039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,3.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3.465909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.465909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3.755556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
ratingDF.T.multiply(simUsers).T

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.437258,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.449367,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.145432,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.076469,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Compute Errors (MAE, RMSE)

In [21]:
userTestRatings = pd.DataFrame(data=test[test['userId'] == userId])
temp = userTestRatings.join(recItemRatings.loc[userTestRatings['movieId']], on='movieId')
mae = (temp['rating'] - temp['prediction']).abs().mean()
rmse = math.sqrt((temp['rating'] - temp['prediction']).pow(2).mean())
print(" MAE:", mae)
print("RMSE:", rmse)

 MAE: 28.3092998391
RMSE: 28.466741109694414


In [22]:
temp

,userId,movieId,rating,prediction
6187,33,1060,4.0,-22.266069
6198,33,1291,4.0,-31.349963
6199,33,1347,2.0,-23.899222
6208,33,1982,4.0,-22.213202
6212,33,2005,4.0,-22.917277
6215,33,2064,5.0,-25.612353
6257,33,3794,4.0,-21.693193
6292,33,4678,3.0,-24.999772
6303,33,4974,3.0,-26.832649
